<a href="https://colab.research.google.com/github/SilvioCesarTeixeira/PJI4/blob/master/PJI4_py_script1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Script base para API

In [ ]:
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch

# Ler o arquivo JSON
with open('dados.json') as f:
    dados = json.load(f)

# Definir o intervalo original de cada variável
pm25_min, pm25_max = 5, 245
o3_min, o3_max = 1, 73
dias_min, dias_max = 44201, 44999

# Criar um objeto MinMaxScaler para cada variável
pm25_scaler = MinMaxScaler(feature_range=(0, 1))
o3_scaler = MinMaxScaler(feature_range=(0, 1))
dias_scaler = MinMaxScaler(feature_range=(0, 1))

# Normalizar o valor de cada variável para o intervalo entre 0 e 1
pm25_normalized = pm25_scaler.fit_transform([[input_dict['PM25']]])[0][0]
o3_normalized = o3_scaler.fit_transform([[input_dict['O3']]])[0][0]
dias_normalized = dias_scaler.fit_transform([[input_dict['Dias']]])[0][0]

# Criar um tensor PyTorch com os valores normalizados
input_tensor = torch.tensor([[pm25_normalized, o3_normalized, dias_normalized]])

# Carregar o modelo LSTM treinado
model = torch.load('modelo_lstm.pt')

# Fazer a predição
output = model(input_tensor)

# Converter a saída para o intervalo desejado
output_min = 0
output_max = 40
output_norm = (output.detach().numpy() - 0) / (1 - 0)
output_final = output_norm * (output_max - output_min) + output_min

print(f"A saída normalizada é: {output_norm:.2f}. O número de internaçãoes é: {output_final:.0f}.")
#print(f'Saída final: {output_final}')

##Script como classe a ser incorporada ao backend 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch

class ModeloLSTM:
    def __init__(self, modelo_path):
        self.pm25_scaler = MinMaxScaler(feature_range=(0, 1))
        self.o3_scaler = MinMaxScaler(feature_range=(0, 1))
        self.dias_scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = torch.load(modelo_path)
        
        # Definir o intervalo original de cada variável
        ## Os valores de mín e máx foram retirados do dataset de treinamento/validação/teste
        self.pm25_min, self.pm25_max = 5, 245 
        self.o3_min, self.o3_max = 1, 73
        self.dias_min, self.dias_max = 44201, 44999
        
        # Definir o intervalo de saída desejado
        self.output_min, self.output_max = 0, 40
    
    def predict(self, df):
        # Normalizar o valor de cada variável para o intervalo entre 0 e 1
        pm25_normalized = self.pm25_scaler.fit_transform(df[["PM25"]])[0][0]
        o3_normalized = self.o3_scaler.fit_transform(df[["O3"]])[0][0]
        dias_normalized = self.dias_scaler.fit_transform(df[["Dias"]])[0][0]
        
        # Criar um tensor PyTorch com os valores normalizados
        input_tensor = torch.tensor([[pm25_normalized, o3_normalized, dias_normalized]])
        
        # Fazer a predição
        output = self.model(input_tensor)
        
        # Converter a saída para o intervalo desejado
        output_norm = (output.detach().numpy() - 0) / (1 - 0)
        output_final = output_norm * (self.output_max - self.output_min) + self.output_min
        
        return output_final

##Testar script 

In [ ]:
modelo = ModeloLSTM("/content/modelo_LSTM_treinado.pt")
dados = pd.DataFrame({"PM25": [100], "O3": [50], "Dias": [44500]})
saida = modelo.predict(dados)
print(f"A saída é: {saida:.0f}")

FileNotFoundError: ignored